In [ ]:
import torch
from preprocessing import get_dataframes
from dataloading import Text_dataset, get_dataloaders
from utilities import set_seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# Enforce reproducability
set_seed(42)

# Create dataframes
train_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/add_prim_split/tasks_train_addprim_turn_left.txt"
test_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/add_prim_split/tasks_test_addprim_turn_left.txt"
train_df, test_df, voc_in, voc_out = get_dataframes(train_url, test_url)

# Create datasets
train_data = Text_dataset(train_df[['IN_idx', 'OUT_idx', 'lens']], sample=True, size=100000)
test_data = Text_dataset(test_df[['IN_idx', 'OUT_idx', 'lens']], sample=False)

# Create
train_loader, test_loader = get_dataloaders(train_data, test_data)

max_len = max(max(test_df.lens),max(train_df.lens))

In [ ]:
import models
from training import train

# Overall best
rnn = 'lstm'
layers = 2
hidden_size = 200
dropout = 0.5

for i in range(5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.DecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len).to(device)      
    train(train_loader, encoder, decoder, device, save_name='ex3_overall_left'+str(i))

In [ ]:
import models
from training import train

# best ex3
rnn = 'gru'
layers = 1
hidden_size = 100
dropout = 0.1   

for i in range(5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.DecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len).to(device)      
    train(train_loader, encoder, decoder, device, save_name='ex3_best_left'+str(i))